In [ ]:
import numpy as np
import tensorflow as tf
import random as python_random
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import random
import importlib

# Seed reset function
def reset_random_seeds(seed_value=42):
    python_random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

reset_random_seeds()

# AES S-box and utility functions
AES_Sbox = [
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
]

# Function to compute expected distortion
def compute_expected_distortion(x, s, Vs, model):
    x = tf.cast(x, tf.float32)
    s = tf.cast(s, tf.float32)
    Vs = tf.cast(Vs, tf.float32)
    perturbed_x = x * s + (1 - s) * Vs
    perturbed_x = tf.reshape(perturbed_x, (1, -1, 1))
    x_reshaped = tf.reshape(x, (1, -1, 1))
    original_pred = model.predict(x_reshaped)
    perturbed_pred = model.predict(perturbed_x)
    return np.linalg.norm(original_pred - perturbed_pred)

# 1-Relaxation with Lagrange Multiplier
def lagrange_multiplier_optimization(theta, lamb, lr, iterations, batch_size, model, x_train, Vs):
    for _ in range(iterations):
        with tf.GradientTape() as tape:
            s = tf.sigmoid(theta)
            batch_loss = 0
            for i in range(batch_size):
                x_i_reshaped = tf.reshape(x_train[i], (trace_length, 1))
                Vs_i_reshaped = tf.reshape(Vs[i], (trace_length, 1))
                s_reshaped = tf.reshape(s, (trace_length, 1))
                batch_loss += compute_expected_distortion(x_i_reshaped, s_reshaped, Vs_i_reshaped, model)
            loss = batch_loss / batch_size + lamb * tf.reduce_sum(s)
        gradients = tape.gradient(loss, [theta])
        theta.assign_sub(lr * gradients[0])

# Hyperparameters for CNN
def get_hyperparameters_cnn(regularization=False):
    hyperparameters = {}
    hyperparameters_mlp = get_hyperparameters_mlp(regularization=regularization, max_dense_layers=3)
    for key, value in hyperparameters_mlp.items():
        hyperparameters[key] = value

    conv_layers = random.choice([1, 2, 3, 4])
    kernels = []
    strides = []
    filters = []
    pooling_types = []
    pooling_sizes = []
    pooling_strides = []
    pooling_type = random.choice(["Average", "Max"])

    for conv_layer in range(1, conv_layers + 1):
        kernel = random.randrange(2, 10, 1)
        kernels.append(kernel)
        strides.append(int(kernel / 2))
        if conv_layer == 1:
            filters.append(random.choice([u for u in range(8, 65, 8)]))
        else:
            filters.append(filters[conv_layer - 2] * 2)
        pool_size = random.choice([2, 3, 4, 5])
        pooling_sizes.append(pool_size)
        pooling_strides.append(pool_size)
        pooling_types.append(pooling_type)

    hyperparameters["conv_layers"] = conv_layers
    hyperparameters["kernels"] = kernels
    hyperparameters["strides"] = strides
    hyperparameters["filters"] = filters
    hyperparameters["pooling_sizes"] = pooling_sizes
    hyperparameters["pooling_strides"] = pooling_strides
    hyperparameters["pooling_types"] = pooling_types

    return hyperparameters

def get_optimizer(optimizer, learning_rate):
    module_name = importlib.import_module("tensorflow.keras.optimizers")
    optimizer_class = getattr(module_name, optimizer)
    return optimizer_class(learning_rate=learning_rate)

def mlp_random(classes, number_of_samples, regularization=False, hp=None):
    hp = get_hyperparameters_mlp(regularization=regularization) if hp is None else hp
    tf_random_seed = np.random.randint(1048576)
    tf.random.set_seed(tf_random_seed)

    inputs = Input(shape=number_of_samples)
    x = None
    for layer_index in range(hp["layers"]):
        if regularization and hp["regularization"] != "dropout":
            x = Dense(hp["neurons"], activation=hp["activation"], kernel_regularizer=get_reg(hp),
                      kernel_initializer=hp["kernel_initializer"],
                      name='dense_{}'.format(layer_index))(inputs if layer_index == 0 else x)
        else:
            x = Dense(hp["neurons"], activation=hp["activation"], kernel_initializer=hp["kernel_initializer"],
                      name='dense_{}'.format(layer_index))(inputs if layer_index == 0 else x)
        if regularization and hp["regularization"] == "dropout":
            x = Dropout(get_reg(hp))(x)

    outputs = Dense(classes, activation='softmax', name='predictions')(x)
    model = Model(inputs, outputs, name='random_mlp')
    optimizer = get_optimizer(hp["optimizer"], hp["learning_rate"])
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model, tf_random_seed, hp

def cnn_random(classes, number_of_samples, regularization=False, hp=None):
    hp = get_hyperparameters_cnn(regularization=regularization) if hp is None else hp
    tf_random_seed = np.random.randint(1048576)
    tf.random.set_seed(tf_random_seed)

    inputs = Input(shape=(number_of_samples, 1))
    x = None
    for layer_index in range(hp["conv_layers"]):
        x = Conv1D(kernel_size=hp["kernels"][layer_index], strides=hp["strides"][layer_index], filters=hp["filters"][layer_index],
                   activation=hp["activation"], padding="same")(inputs if layer_index == 0 else x)
        if hp["pooling_types"][layer_index] == "Average":
            x = AveragePooling1D(pool_size=hp["pooling_sizes"][layer_index], strides=hp["pooling_strides"][layer_index], padding="same")(x)
        else:
            x = MaxPooling1D(pool_size=hp["pooling_sizes"][layer_index], strides=hp["pooling_strides"][layer_index], padding="same")(x)
        x = BatchNormalization()(x)
    x = Flatten()(x)

    for layer_index in range(hp["layers"]):
        if regularization and hp["regularization"] != "dropout":
            x = Dense(hp["neurons"], activation=hp["activation"], kernel_regularizer=get_reg(hp),
                      kernel_initializer=hp["kernel_initializer"], name='dense_{}'.format(layer_index))(x)
        else:
            x = Dense(hp["neurons"], activation=hp["activation"], kernel_initializer=hp["kernel_initializer"],
                      name='dense_{}'.format(layer_index))(x)
        if regularization and hp["regularization"] == "dropout":
            x = Dropout(hp["dropout"])(x)

    outputs = Dense(classes, activation='softmax', name='predictions')(x)
    model = Model(inputs, outputs, name='random_cnn')
    optimizer = get_optimizer(hp["optimizer"], hp["learning_rate"])
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model, tf_random_seed, hp

# Function for Guessing Entropy (GE) and Correlation Power Analysis (CPA)
def perform_attacks(nb_traces=1000, predictions=None, plt_attack=None, correct_key=None, leakage="HW", dataset=None, nb_attacks=100, shuffle=True):
    all_rk_evol = np.zeros((nb_attacks, nb_traces))
    all_key_log_prob = np.zeros(256)
    for i in tqdm(range(nb_attacks)):
        if shuffle:
            l = list(zip(predictions, plt_attack))
            random.shuffle(l)
            sp, splt = list(zip(*l))
            sp = np.array(sp)
            splt = np.array(splt)
            att_pred = sp[:nb_traces]
            att_plt = splt[:nb_traces]
        else:
            att_pred = predictions[:nb_traces]
            att_plt = plt_attack[:nb_traces]
        rank_evol, key_log_prob = rank_compute(att_pred, att_plt, correct_key, leakage=leakage, dataset=dataset)
        all_rk_evol[i] = rank_evol
        all_key_log_prob += key_log_prob
    return np.mean(all_rk_evol, axis=0), key_log_prob

def rank_compute(prediction, att_plt, correct_key, leakage, dataset):
    hw = [bin(x).count("1") for x in range(256)]
    (nb_traces, nb_hyp) = prediction.shape
    key_log_prob = np.zeros(256)
    prediction = np.log(prediction + 1e-40)
    rank_evol = np.full(nb_traces, 255)
    for i in range(nb_traces):
        for k in range(256):
            att_byte = int(att_plt[i, 0])
            if dataset == "AES_HD_ext":
                if leakage == 'ID':
                    key_log_prob[k] += prediction[i, AES_Sbox_inv[k ^ att_byte] ^ att_byte]
                else:
                    key_log_prob[k] += prediction[i, hw[AES_Sbox_inv[k ^ att_byte] ^ att_byte]]
            elif dataset == "AES_HD_ext_ID":
                if leakage == 'ID':
                    key_log_prob[k] += prediction[i, AES_Sbox_inv[k ^ att_byte]]
                else:
                    key_log_prob[k] += prediction[i, hw[AES_Sbox_inv[k ^ att_byte]]]
            else:
                if leakage == 'ID':
                    key_log_prob[k] += prediction[i, AES_Sbox[k ^ att_byte]]
                else:
                    key_log_prob[k] += prediction[i, hw[AES_Sbox[k ^ att_byte]]]
        rank_evol[i] = rk_key(key_log_prob, correct_key)
    return rank_evol, key_log_prob

def rk_key(rank_array, key):
    key_val = rank_array[key]
    final_rank = np.float32(np.where(np.sort(rank_array)[::-1] == key_val)[0][0])
    if math.isnan(float(final_rank)) or math.isinf(float(final_rank)):
        return np.float32(256)
    else:
        return np.float32(final_rank)

def NTGE_fn(GE):
    NTGE = float('inf')
    for i in range(GE.shape[0] - 1, -1, -1):
        if GE[i] > 0:
            NTGE = i
            break
    return NTGE

def aes_label_cpa(plaintexts, correct_key, leakage):
    num_traces = plaintexts.shape[0]
    labels_for_snr = np.zeros(num_traces)
    for i in range(num_traces):
        if leakage == 'HW':
            labels_for_snr[i] = HW(AES_Sbox[plaintexts[i, 0] ^ correct_key])
        elif leakage == 'ID':
            labels_for_snr[i] = AES_Sbox[plaintexts[i, 0] ^ correct_key]
    return labels_for_snr

def perform_cpa_all_keys(traces, plaintexts, num_keys=256):
    num_traces, num_samples = traces.shape
    correlations = np.zeros((num_keys, num_samples))
    for key_guess in tqdm(range(num_keys)):
        labels_for_cpa = aes_label_cpa(plaintexts, key_guess, leakage)
        for t in range(num_samples):
            correlations[key_guess, t] = abs(np.corrcoef(labels_for_cpa[:num_traces], traces[:, t])[1, 0])
    return correlations

def plot_correlations(correlations, correct_key):
    num_keys, num_samples = correlations.shape
    plt.figure(figsize=(10, 8))
    for key_guess in range(num_keys):
        if key_guess != correct_key:
            plt.plot(correlations[key_guess], color='grey', label='Other Keys' if 'Other Keys' not in plt.gca().get_legend_handles_labels()[1] else "", alpha=0.5, linewidth=1)
    plt.plot(correlations[correct_key], color='blue', label=f'Correct Key: {correct_key:02X}', linewidth=2)
    plt.title('Correlation Power Analysis Across All Key Guesses')
    plt.xlabel('Sample Points')
    plt.ylabel('Absolute Correlation')
    plt.grid(True)
    plt.legend(loc='upper right')
    plt.show()

if __name__ == "__main__":
    # Data loading and preprocessing
    num_traces = 10000
    trace_length = 5000
    leakage = "ID"
    chipwhisper_folder = '/home/localuserplr/Documents/FYP/chipwhisperer/'

    X = np.load(chipwhisper_folder + 'traces.npy')[:num_traces]
    y = np.load(chipwhisper_folder + 'labels.npy')[:num_traces]
    plaintexts = np.load(chipwhisper_folder + 'plain.npy')[:num_traces]
    keys = np.load(chipwhisper_folder + 'key.npy')[:num_traces]

    X = X.reshape((X.shape[0], X.shape[1], 1))

    X_train_val, X_attack, y_train_val, y_attack, plaintexts_train_val, plaintexts_attack = train_test_split(
        X, y, plaintexts, test_size=0.2, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

    y_train_categorical = tf.keras.utils.to_categorical(y_train, num_classes=256)
    y_val_categorical = tf.keras.utils.to_categorical(y_val, num_classes=256)

    classes = 256
    number_of_samples = trace_length
    model_type = "cnn"
    regularization = True
    
    # Parameters for perturbation
    k = trace_length
    lamb = 0.1
    num_iterations = 10
    learning_rate = 0.01
    batch_size = 32

    # Initialize theta for 1-Relaxation with Lagrange Multiplier
    theta = tf.Variable(np.ones(k) * 0.5, dtype=tf.float32)

    # Optimizer
    optimizer = tf.optimizers.Adam(learning_rate)

    # Initialize model
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=(trace_length, 1)),
        Flatten(),
        Dense(256, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Random data (example)
    x = X_train[:batch_size]  # Example batch
    Vs = np.random.randn(batch_size, k)

    # Optimization loop for feature selection using 1-Relaxation with Lagrange Multiplier
    lagrange_multiplier_optimization(theta, lamb, learning_rate, num_iterations, batch_size, model, x, Vs)

    # Resulting mask
    s = tf.sigmoid(theta).numpy()

    # Apply the mask to the training data
    X_train_masked = X_train * s[np.newaxis, :, np.newaxis]
    X_val_masked = X_val * s[np.newaxis, :, np.newaxis]

    # Function to create and train models
    def create_and_train_model(x_train, y_train, x_val, y_val, model_type="cnn", regularization=True):
        classes = 256
        number_of_samples = x_train.shape[1]
        if model_type == "mlp":
            hp = get_hyperparameters_mlp(regularization=regularization)
            model, seed, hp = mlp_random(classes, number_of_samples, hp=hp, regularization=regularization)
        else:
            hp = get_hyperparameters_cnn(regularization=regularization)
            model, seed, hp = cnn_random(classes, number_of_samples, hp=hp, regularization=regularization)

        history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), batch_size=hp["batch_size"])
        return model, history

    # Function to plot accuracy
    def plot_accuracy(history, title):
        plt.figure(figsize=(10, 6))
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(title)
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)
        plt.show()

    # Train and evaluate models with and without 1-Relaxation with Lagrange Multiplier
    models = []
    histories_with_lagrange = []
    histories_without_lagrange = []

    # With 1-Relaxation with Lagrange Multiplier feature selection
    for i in range(5):
        try:
            model, history = create_and_train_model(X_train_masked, y_train_categorical, X_val_masked, y_val_categorical, model_type="cnn", regularization=True)
            models.append(model)
            histories_with_lagrange.append(history)

            print(f"Model {i+1} summary (with 1-Relaxation with Lagrange Multiplier):")
            model.summary()

            predictions = model.predict(X_attack)
            predicted_classes = np.argmax(predictions, axis=1)
            accuracy = np.mean(predicted_classes == y_attack)
            print(f"Test set accuracy for model {i+1} (with 1-Relaxation with Lagrange Multiplier): {accuracy}")

            plot_accuracy(history, f'Model {i+1} Accuracy (with 1-Relaxation with Lagrange Multiplier)')
        except Exception as e:
            print(f"An error occurred while creating model {i+1} (with 1-Relaxation with Lagrange Multiplier): {e}")

    # Without 1-Relaxation with Lagrange Multiplier feature selection
    for i in range(5):
        try:
            model, history = create_and_train_model(X_train, y_train_categorical, X_val, y_val_categorical, model_type="cnn", regularization=True)
            models.append(model)
            histories_without_lagrange.append(history)

            print(f"Model {i+1} summary (without 1-Relaxation with Lagrange Multiplier):")
            model.summary()

            predictions = model.predict(X_attack)
            predicted_classes = np.argmax(predictions, axis=1)
            accuracy = np.mean(predicted_classes == y_attack)
            print(f"Test set accuracy for model {i+1} (without 1-Relaxation with Lagrange Multiplier): {accuracy}")

            plot_accuracy(history, f'Model {i+1} Accuracy (without 1-Relaxation with Lagrange Multiplier)')
        except Exception as e:
            print(f"An error occurred while creating model {i+1} (without 1-Relaxation with Lagrange Multiplier): {e}")

    # Compare the average accuracy of models with and without 1-Relaxation with Lagrange Multiplier feature selection
    if histories_with_lagrange:
        avg_train_acc_with_lagrange = np.mean([history.history['accuracy'][-1] for history in histories_with_lagrange])
        avg_val_acc_with_lagrange = np.mean([history.history['val_accuracy'][-1] for history in histories_with_lagrange])
    else:
        avg_train_acc_with_lagrange = np.nan
        avg_val_acc_with_lagrange = np.nan

    if histories_without_lagrange:
        avg_train_acc_without_lagrange = np.mean([history.history['accuracy'][-1] for history in histories_without_lagrange])
        avg_val_acc_without_lagrange = np.mean([history.history['val_accuracy'][-1] for history in histories_without_lagrange])
    else:
        avg_train_acc_without_lagrange = np.nan
        avg_val_acc_without_lagrange = np.nan

    print(f"Average training accuracy with 1-Relaxation with Lagrange Multiplier: {avg_train_acc_with_lagrange}")
    print(f"Average validation accuracy with 1-Relaxation with Lagrange Multiplier: {avg_val_acc_with_lagrange}")
    print(f"Average training accuracy without 1-Relaxation with Lagrange Multiplier: {avg_train_acc_without_lagrange}")
    print(f"Average validation accuracy without 1-Relaxation with Lagrange Multiplier: {avg_val_acc_without_lagrange}")

    # Plot comparison of average accuracy
    labels = ['Training Accuracy', 'Validation Accuracy']
    with_lagrange = [avg_train_acc_with_lagrange, avg_val_acc_with_lagrange]
    without_lagrange = [avg_train_acc_without_lagrange, avg_val_acc_without_lagrange]

    x = np.arange(len(labels))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, with_lagrange, width, label='With 1-Relaxation with Lagrange Multiplier')
    rects2 = ax.bar(x + width/2, without_lagrange, width, label='Without 1-Relaxation with Lagrange Multiplier')

    ax.set_ylabel('Accuracy')
    ax.set_title('Average Accuracy with and without 1-Relaxation with Lagrange Multiplier Feature Selection')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()
    plt.grid(True)
    plt.show()
